In [ ]:
import numpy as np
import pickle
from random import choices
import pandas as pd
from dataset_helpers import *

# Structures

In [ ]:
with open('/houses/training_data.pkl','rb') as f:
    training_data = pickle.load(f)
    
with open('/houses/validation_data.pkl','rb') as f:
    validation_data = pickle.load(f)

data = []
for i in range(len(training_data)):
    data.append((training_data[i][1], training_data[i][2]))
for i in range(len(validation_data)):
    data.append((validation_data[i][1], validation_data[i][2]))

In [ ]:
# testing the annotation reversal
a = [[[1,1,2], [3,4,5], [6,7,8]]]
b = ['nothing', 'roof', 'windows', 'windows', 'windows', 'windows', 'windows', 'none', 'wall' ]

reversed_annotation = b[1:] # exclude 'nothing' from re-indexing
reversed_annotation.reverse()
resorted_annotation = ['nothing']
resorted_annotation += reversed_annotation

print(clean_segmentation(np.array(a), b))
print(resorted_annotation)

[[[8 8 0]
  [0 0 0]
  [0 0 1]]]
['nothing', 'wall', 'none', 'windows', 'windows', 'windows', 'windows', 'windows', 'roof']


In [ ]:
structures = []
for (segmentation, annotation) in data:
    structure = clean_segmentation(segmentation, annotation)
    reversed_annotation = annotation[1:] # exclude 'nothing' tag from reversal
    reversed_annotation.reverse()
    resorted_annotation = ['nothing']
    resorted_annotation += reversed_annotation
    structures.append((structure, resorted_annotation))
    for n in range(1, 4):
        structures.append((np.rot90(structure, k=n, axes=(0, 2)), resorted_annotation))

In [ ]:
with open('structures.pkl', 'wb') as f:
    pickle.dump(structures, f, protocol=pickle.DEFAULT_PROTOCOL)

# Markov Chains

In [ ]:
with open('structures.pkl','rb') as f:
    structures = pickle.load(f)

In [ ]:
transition_table, segments = calculate_markov_transitions(structures)

In [ ]:
with open('transition_table.pkl', 'wb') as f:
    pickle.dump(transition_table, f, protocol=pickle.DEFAULT_PROTOCOL)

with open('segments_dict.pkl', 'wb') as f:
    pickle.dump(segments, f, protocol=pickle.DEFAULT_PROTOCOL)

In [ ]:
import pickle 
with open('transition_table.pkl', 'wb') as f:
    pickle.dump(transition_table, f, protocol=pickle.DEFAULT_PROTOCOL)a

# Data Generation

In [ ]:
with open('transition_table.pkl','rb') as f:
    transition_table = pickle.load(f)

with open('segments_dict.pkl','rb') as f:
    segments_dict = pickle.load(f)

In [ ]:
DIM = 18
down = ['floor', 'base', 'bottom', 'ground', 'foundation']
up = ['roof', 'top', 'layer', 'ceiling', 'ledge', 'overhang', 'platform']

def get_y_indeces(structure, x, z, segment, name):
    s0, s1, s2 = segment.shape

    for word in down:
        # floor types go to the bottom
        if name.find(word) != -1:
            return 0, s1

    for word in up:
        # roof types go to the top
        if name.find(word) != -1:
            # roofs must not float
            # find highest point in designated area
            non_zeroIds = np.nonzero(
                # slice the requested area
                structure[x:x+s0, 0:DIM, z:z+s2])
            # use the highest non-zero y index
            if len(non_zeroIds[1]) == 0:
                # roof part falls to the ground
                return 1, 1+s1
            # prevent building from clipping OOB vertically
            h = non_zeroIds[1].max()
            safetyClip = h + s1 - DIM if h + s1 >= DIM else 0
            return (h - safetyClip, h + s1 - safetyClip)

    # wall types stand upright on the floor
    return 1, 1+s1

def generate_structure(annotation, segmentsDict):
    structure = np.zeros(shape=(DIM,DIM,DIM), dtype=np.uint8)
    # start building from the outer thirds inward
    outer_thirds = [i for i in range(int(DIM/3))] + [i for i in range(int(2*DIM/3), DIM)]
    [x, z] = choices(outer_thirds, k=2)
    y = 0 # vertical dim in minetest

    # in the outer thirds of the space we invert horizontal directions
    turn_positive_zone = list(range(int(DIM/3)))
    turn_negative_zone = [2 * int(DIM/3) + i for i in range(int(DIM/3))]
    x_dir = 1 if x in turn_positive_zone else -1
    z_dir = 1 if z in turn_positive_zone else -1

    for segment_idx, segment_name in enumerate(annotation):
        if segment_idx == 0 or segment_name == "Done":
            continue
        segment = choices(segmentsDict[segment_name])[0]
        segment[segment == 1] = segment_idx
        s0, s1, s2 = segment.shape

        x_dir = 1 if x in turn_positive_zone else x_dir
        z_dir = 1 if z in turn_positive_zone else z_dir
        x_dir = -1 if x in turn_negative_zone else x_dir
        z_dir = -1 if z in turn_negative_zone else z_dir
        # some safety constraints
        x_dir *= -1 if x+x_dir*s0 >= DIM or x+x_dir*s0 < 0 else 1
        z_dir *= -1 if z+z_dir*s2 >= DIM or z+z_dir*s2 < 0 else 1
        next_x = x+x_dir*s0
        next_z = z+z_dir*s2
        
        # update vertical position according to segment type
        y, next_y = get_y_indeces(structure, x, z, segment, segment_name)
        
        # clip segments which do not fit in either direction 
        diff = [0, 0, 0]
        for i, s in enumerate(structure[x:next_x:x_dir, y:next_y, z:next_z:z_dir].shape):
            diff[i] = s - segment.shape[i]
        if diff != [0, 0, 0]:
            segment = segment[0:s0+diff[0], 0:s1+diff[1], 0:s2+diff[2]]
        
        try:
            structure[x:next_x:x_dir, y:next_y, z:next_z:z_dir] = segment
        except Exception as e:
            print('space:', structure[x:next_x:x_dir, y:next_y, z:next_z:z_dir].shape)
            print('segment:', segment.shape)
            print('diff:', diff)

        # build along the same axis
        if s0 > s2:
            x = next_x
        else:
            z = next_z
    
    # position the structure randomly to make it more easily
    # distinguishable from noise than dense structures
    padded_structure = np.zeros((24, 24, 24), dtype=np.uint8)
    [rx, rz] = choices(list(range(6)), k=2)
    padded_structure[rx:rx+DIM, 0:+DIM, rz:rz+DIM] = structure
    return padded_structure

# testing that everything works as expected
markov_annotation = generate_annotation(transition_table, 20)
print(markov_annotation)
artificial_structure = generate_structure(markov_annotation, segments_dict)
print(artificial_structure.shape)

['Start', 'house support', 'house support', 'house support', 'house support', 'house support', 'floor', 'pillar', 'pillar', 'pillar', 'pillar', 'pillar', 'pillar', 'pillar', 'roof', 'roof', 'roof', 'roof', 'roof', 'floor', 'wall', 'roof', 'Done']
(24, 24, 24)


In [ ]:
from tqdm import tqdm
artificial_structures = []
for i in tqdm(range(100000)):
    a = generate_annotation(transition_table)
    artificial_structures.append((generate_structure(a, segments_dict), a))

100%|██████████| 100000/100000 [01:32<00:00, 1079.74it/s]


In [ ]:
with open('../experiments/artificial_structures.pkl', 'wb') as f:
    pickle.dump(artificial_structures, f, protocol=pickle.DEFAULT_PROTOCOL)

In [19]:
import pickle
import numpy as np
import torch
with open('../experiments/data/artificial_structures.pkl','rb') as f:
    d = pickle.load(f)
# scale input down to [0, 1]
data = np.array([M[0] for M in d])

In [20]:
data = data.astype(np.float32) / 125.5 - 1

print(data.min())
print(data.max())
print(data.shape, data.dtype)

-1.0
1.0
(100000, 24, 24, 24) float32


In [3]:
with open('../experiments/segmentation_3d_data.pkl', 'wb') as f:
    pickle.dump(data, f, protocol=pickle.DEFAULT_PROTOCOL)

In [1]:
import pickle
import numpy as np
import torch
with open('/data/segmentation_3d_data.pkl','rb') as f:
    d = pickle.load(f)

data = torch.from_numpy(d)
print(d.size())
print(d.dtype)
print(d.min())
print(d.max())

torch.Size([100000, 24, 24, 24])
torch.float32
tensor(-1.)
tensor(1.)
